In [1]:
import torch
#################################### For Image ####################################
from PIL import Image
from sam3.model_builder import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor
# Load the model
model = build_sam3_image_model()
processor = Sam3Processor(model)
# Load an image
image = Image.open("/home/pierre/Downloads/proxy-image.jpg")
inference_state = processor.set_image(image)
# Prompt the model with text
output = processor.set_text_prompt(state=inference_state, prompt="drone")

# Get the masks, bounding boxes, and scores
masks, boxes, scores = output["masks"], output["boxes"], output["scores"]


/home/pierre/Documents/Projects/PST4/AI/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pierre/Documents/Projects/PST4/AI/.venv/lib/python3.13/site-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


In [1]:
from transformers import Sam3TrackerVideoModel, Sam3TrackerVideoProcessor, Sam3VideoProcessor
from accelerate import Accelerator
import torch

/home/pierre/Documents/Projects/PST4/AI/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = Accelerator().device
model = Sam3TrackerVideoModel.from_pretrained("facebook/sam3").to(device, dtype=torch.bfloat16)
processor = Sam3VideoProcessor.from_pretrained("facebook/sam3")

Loading weights: 1494it [00:01, 1530.08it/s, Materializing param=detector_model.detr_encoder.layers.2.cross_attn.q_proj.bias]                                               IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [3]:
from transformers.video_utils import load_video
video_url = "https://huggingface.co/datasets/hf-internal-testing/sam2-fixtures/resolve/main/bedroom.mp4"
video_frames, _ = load_video(video_url)

In [4]:
inference_session = processor.init_video_session(
    inference_device=device,
    dtype=torch.bfloat16,
)

In [6]:
for frame_idx, frame in enumerate(video_frames[:10]):  # Process first 10 frames
    inputs = processor(images=frame, device=device, return_tensors="pt", text="bed")

    # sam3_tracker_video_output = model(inference_session=inference_session, frame=inputs.pixel_values[0])
    # video_res_masks = processor.post_process_masks(
    #     [sam3_tracker_video_output.pred_masks], original_sizes=inputs.original_sizes, binarize=False
    # )[0]
    # print(f"Frame {frame_idx}: mask shape {video_res_masks.shape}")


Unused or unrecognized kwargs: text.


TypeError: Sam3FastImageProcessorKwargs.__init__() got an unexpected keyword argument 'text'

In [8]:
import cv2
from PIL import Image
from sam3.model_builder import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor

model = build_sam3_image_model()
processor = Sam3Processor(model)

cap = cv2.VideoCapture(0)

fps = 30
interval_frames = int(fps * 0.5)

frame_index = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    print("")
    if frame_index % interval_frames == 0:
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        state = processor.set_image(pil_image)
        out = processor.set_text_prompt(state=state, prompt="drone")
        masks, boxes, scores = out["masks"], out["boxes"], out["scores"]
        print("Hello")
    frame_index += 1

cap.release()


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 7.60 GiB of which 33.69 MiB is free. Process 35858 has 61.97 MiB memory in use. Including non-PyTorch memory, this process has 6.34 GiB memory in use. Of the allocated memory 6.01 GiB is allocated by PyTorch, and 179.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)